In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from django.db.models import F

In [2]:
# Set the run id and query id
run_id=181
qid=1281

In [30]:
dts = DocTopic.objects.filter(run_id=run_id).values('doc_id','doc__title','topic__title','score')
df = pd.DataFrame.from_dict(list(dts)).pivot_table(index=['doc_id','doc__title'],columns='topic__title',values='score').fillna(0)
df.head()
df.to_csv('../data/doc')

,topic__title,"{activ, school, health}","{air, pollut, qualiti}","{bicycl, cycl, cyclist}","{brt, bus, system}","{bus, servic, oper}","{climat, chang, mitig}","{develop, sustain, plan}","{emiss, carbon, ghg}","{energi, build, consumpt}","{green, build, roof}",...,"{recycl, household, collect}","{toll, lane, link}","{transit, rail, station}","{transport, public, access}","{travel, mode, choic}","{treatment, wastewat, plant}","{urban, land, citi}","{vehicl, electr, charg}","{wast, manag, solid}","{water, heat, suppli}"
doc_id,doc__title,,,,,,,,,,,,,,,,,,,,,
WOS:000071014200004,Principle of marginal-cost pricing: How does it work in a general road network?,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.014760,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.007942,0.0,0.000000,0.000000,0.000000,0.0
WOS:000071108200001,The rise and fall of the American carpool: 1970-1990,0.026979,0.00000,0.0,0.0,0.000000,0.002095,0.000000,0.0,0.0,0.0,...,0.010091,0.000000,0.0,0.005345,0.031565,0.0,0.013904,0.013002,0.000000,0.0
WOS:000071124600002,Microscopic simulation of urban traffic based on cellular automata,0.000000,0.01246,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.008466,0.000000,0.0,0.013452,0.003693,0.000000,0.0
WOS:000071225300004,"General equilibrium vs. optimum, and the allocation of land for transportation in a closed information-oriented city with traffic congestion",0.000000,0.00000,0.0,0.0,0.005947,0.000000,0.006832,0.0,0.0,0.0,...,0.002689,0.000000,0.0,0.005747,0.005744,0.0,0.012986,0.000000,0.000000,0.0
WOS:000071306300006,Experiences from Singapore's Park-and-Ride Scheme (1975-1996),0.000000,0.00000,0.0,0.0,0.006773,0.000000,0.006541,0.0,0.0,0.0,...,0.000000,0.001436,0.0,0.000000,0.011421,0.0,0.000000,0.010510,0.001967,0.0


In [17]:
# Get the papers
papers = Doc.objects.filter(query=qid)[:5]
for p in papers:
    p.main_topics = p.doctopic_set.filter(run_id=run_id,score__gt=0).order_by('-score')

In [18]:
print(papers[0].title)
print([str(x.score) + "  :  " + x.topic.title  for x in papers[0].main_topics])


Effects of aggregate reuse for overpass reconstruction-extension projects on energy conservation and greenhouse gas reduction: A case study from Shanghai City
['0.0524018090338123  :  {emiss, carbon, ghg}', '0.0276011659810759  :  {energi, build, consumpt}', '0.017229809438139  :  {bicycl, cycl, cyclist}', '0.0104287121137904  :  {develop, sustain, plan}', '0.00689345877789529  :  {urban, land, citi}', '0.00678621957804073  :  {wast, manag, solid}', '0.00451707505688004  :  {travel, mode, choic}']
